1. 라이브러리 및 데이터

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats

In [2]:
# load data
train = pd.read_csv('C:/Users/zz325/PycharmProjects/2020742043/Datamining/1.Training/원천데이터/2.수면/train_sleep.csv')
train_label = pd.read_csv('C:/Users/zz325/PycharmProjects/2020742043/Datamining/1.Training/라벨링데이터/2.수면/training_label.csv')
test = pd.read_csv('C:/Users/zz325/PycharmProjects/2020742043/Datamining/2.Validation/원천데이터/2.수면/val_sleep.csv')
test_label = pd.read_csv('C:/Users/zz325/PycharmProjects/2020742043/Datamining/2.Validation/라벨링데이터/2.수면/val_label.csv')


In [3]:
# train data 연구 참여자 수: 141 = 85(CN) + 47(MCI) + Dem(9)
# test data 연구 참여자 수: 33 = 26(CN) + MCI(4) + Dem(3)

print(train.shape)
print(train_label.shape)
print(test.shape)
print(test_label.shape)
print(train_label['DIAG_NM'].value_counts())
print(test_label['DIAG_NM'].value_counts())

(9705, 36)
(141, 2)
(2478, 36)
(33, 2)
DIAG_NM
CN     85
MCI    47
Dem     9
Name: count, dtype: int64
DIAG_NM
CN     26
MCI     4
Dem     3
Name: count, dtype: int64


In [6]:
def preprocessing(train, label):
    drop_cols = train.describe(include = 'O').columns[1:]
    train = train.drop(drop_cols, axis = 1)
    data = train.groupby('EMAIL').mean().reset_index()
    labeling = dict(np.array(label))

    data['target'] = data['EMAIL'].map(labeling)
    train['target'] = train['EMAIL'].map(labeling)

    # target 변수 numerical data로 변환
    target_dict = {"CN": 1, "MCI": 2, "Dem": 3}
    data['target'] = data['target'].map(target_dict)
    data = data.sort_values(by=['target','EMAIL'], ascending=[True,True])
    data = data.reset_index(drop=True)
    
    train['target'] = train['target'].map(target_dict)
    train = train.sort_values(by=['target','EMAIL'], ascending=[True,True])
    train = train.reset_index(drop=True)

    # Step 1: 'number' 열을 생성 (EMAIL에서 숫자만 추출)
    data['number'] = data['EMAIL'].str.extract(r'(?<=nia\+)(\d+)', expand=False).astype(int)
    train['number'] = train['EMAIL'].str.extract(r'(?<=nia\+)(\d+)', expand=False).astype(int)

    # Step 2: EMAIL 열 삭제
    data.drop(columns=['EMAIL'], inplace=True)
    train.drop(columns=['EMAIL'], inplace=True)

    return data, train

train_mean, train_all = preprocessing(train, train_label)
train_all

,sleep_awake,sleep_breath_average,sleep_deep,sleep_duration,sleep_efficiency,sleep_hr_average,sleep_hr_lowest,sleep_is_longest,sleep_light,sleep_midpoint_at_delta,...,sleep_score_disturbances,sleep_score_efficiency,sleep_score_latency,sleep_score_rem,sleep_score_total,sleep_temperature_delta,sleep_temperature_deviation,sleep_total,target,number
0,1860,15.875,5640,24480,92,59.63,54,1,11670,10776,...,79,97,94,72,61,-0.13,-0.13,22620,1,5
1,1860,16.000,9660,31200,94,60.74,53,1,11250,8204,...,80,99,99,99,88,0.11,0.11,29340,1,5
2,3210,15.625,7230,30420,89,61.25,53,1,14100,8574,...,79,95,97,79,79,-0.10,-0.10,27210,1,5
3,1890,16.250,6690,28980,93,60.26,53,1,15750,9217,...,81,99,91,63,79,0.13,0.13,27090,1,5
4,1200,15.875,4980,30180,96,60.71,55,1,18600,8349,...,84,100,72,73,86,-0.41,-0.41,28980,1,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9700,9480,15.625,3720,46200,79,66.78,60,1,20580,6719,...,56,72,94,100,100,0.41,0.41,36720,3,341
9701,8280,15.000,3480,38280,78,70.26,62,1,18030,3181,...,62,67,99,99,96,0.07,0.07,30000,3,341
9702,2580,17.625,4890,26940,90,69.55,63,1,13470,10501,...,60,96,72,90,74,-0.14,-0.14,24360,3,341
9703,8940,16.625,4470,40860,78,70.85,63,1,18420,5415,...,63,67,97,100,99,0.25,0.25,31920,3,341


In [5]:
print(len(train_all.columns))

29


In [11]:
def feature_by_pvalue(df):
    independent_vars = list(df.columns)

    anova_results = {}

    for var in independent_vars:
        group0 = df[df['target'] == 1][var]
        group1 = df[df['target'] == 2][var]
        group2 = df[df['target'] == 3][var]
    
        f_val, p_val = stats.f_oneway(group0, group1, group2)
        anova_results[var] = p_val

    sorted_vars = sorted(anova_results, key=anova_results.get)

    significant_vars = [var for var in sorted_vars if anova_results[var] < 0.05]

    print("유의미한 변수들:", significant_vars)
    print(len(significant_vars))

    return significant_vars

In [12]:
data_for_feature_all = train_all
feature_by_pvalue(data_for_feature_all)

유의미한 변수들: ['target', 'number', 'sleep_light', 'sleep_restless', 'sleep_midpoint_time', 'sleep_duration', 'sleep_total', 'sleep_score_deep', 'sleep_score_alignment', 'sleep_awake', 'sleep_deep', 'sleep_score_total', 'sleep_onset_latency', 'sleep_midpoint_at_delta', 'sleep_score_latency', 'sleep_breath_average', 'sleep_hr_lowest', 'sleep_score_efficiency', 'sleep_efficiency', 'sleep_rem', 'sleep_score_disturbances', 'sleep_hr_average', 'sleep_period_id', 'sleep_score_rem', 'sleep_rmssd', 'sleep_temperature_delta', 'sleep_temperature_deviation']
27


c:\anaconda3\envs\datamining\Lib\site-packages\scipy\stats\_axis_nan_policy.py:531: ConstantInputWarning: Each of the input arrays is constant; the F statistic is not defined or infinite
  res = hypotest_fun_out(*samples, **kwds)


['target',
 'number',
 'sleep_light',
 'sleep_restless',
 'sleep_midpoint_time',
 'sleep_duration',
 'sleep_total',
 'sleep_score_deep',
 'sleep_score_alignment',
 'sleep_awake',
 'sleep_deep',
 'sleep_score_total',
 'sleep_onset_latency',
 'sleep_midpoint_at_delta',
 'sleep_score_latency',
 'sleep_breath_average',
 'sleep_hr_lowest',
 'sleep_score_efficiency',
 'sleep_efficiency',
 'sleep_rem',
 'sleep_score_disturbances',
 'sleep_hr_average',
 'sleep_period_id',
 'sleep_score_rem',
 'sleep_rmssd',
 'sleep_temperature_delta',
 'sleep_temperature_deviation']

In [13]:
data_for_feature_mean = train_mean
feature = feature_by_pvalue(data_for_feature_mean)

유의미한 변수들: ['target', 'number', 'sleep_light', 'sleep_score_latency', 'sleep_midpoint_time', 'sleep_duration', 'sleep_onset_latency', 'sleep_restless', 'sleep_score_alignment', 'sleep_total', 'sleep_score_deep', 'sleep_awake']
12


c:\anaconda3\envs\datamining\Lib\site-packages\scipy\stats\_axis_nan_policy.py:531: ConstantInputWarning: Each of the input arrays is constant; the F statistic is not defined or infinite
  res = hypotest_fun_out(*samples, **kwds)


모델적용

In [14]:
X_mean = train_mean
X_mean.drop(columns=['number'], inplace=True)
X_mean_target = X_mean['target']
X_mean.drop(columns=['target'], inplace=True)

In [15]:
print(X_mean.values.shape)

(141, 27)


In [ ]:
# significant value list에 number와 target이 포함되어 있어 삭제
feature.remove('number')
feature.remove('target')

In [17]:
data_for_logit = X_mean[feature].copy()
data_for_logit


,sleep_light,sleep_score_latency,sleep_midpoint_time,sleep_duration,sleep_onset_latency,sleep_restless,sleep_score_alignment,sleep_total,sleep_score_deep,sleep_awake
0,16303.539823,83.973451,15127.699115,30115.752212,672.212389,29.300885,86.849558,28060.088496,89.345133,2055.663717
1,14088.000000,74.800000,14404.285714,28422.857143,578.571429,33.571429,89.028571,23275.714286,96.542857,5147.142857
2,12606.891892,79.905405,13722.972973,27224.594595,528.243243,32.202703,89.500000,21789.324324,99.081081,5435.270270
3,14666.202532,84.000000,17314.556962,34593.417722,817.974684,29.696203,95.430380,25923.037975,92.063291,8670.379747
4,18325.890411,83.945205,14871.780822,30269.589041,866.712329,43.602740,99.616438,24827.671233,96.904110,5441.917808
...,...,...,...,...,...,...,...,...,...,...
136,14092.500000,81.931818,14036.590909,26896.363636,991.363636,51.931818,89.704545,19431.136364,56.909091,7465.227273
137,19578.620690,68.609195,18986.896552,36944.827586,1412.413793,27.850575,99.942529,32936.206897,96.919540,4008.620690
138,20261.756757,72.878378,15657.972973,31551.081081,459.729730,46.729730,89.216216,27824.189189,48.081081,3726.891892
139,14174.262295,81.983607,12480.491803,24405.245902,641.311475,41.360656,97.737705,21195.245902,94.377049,3210.000000


In [18]:
# 다중 로지스틱 회귀 구현
import torch
import torch.nn as nn

class DementiaNet(nn.Module):
    def __init__(self):
        super(DementiaNet, self).__init__()
        self.fc1 = nn.Linear(10, 256)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(256, 3)
        
    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x
    
x = torch.tensor(data_for_logit.values).float()
y = torch.tensor(X_mean_target.values - 1).long()

In [19]:
net = DementiaNet()
opt = torch.optim.Adam(net.parameters(), lr=1e-3)
los = nn.CrossEntropyLoss()

In [20]:
from collections import defaultdict
 
bs = 16
for e in range(1,21):
    eloss = defaultdict(lambda: [])
    for idx in range(0,141,bs):
        opt.zero_grad()
        p = net(x[idx:idx+bs])
        loss = los(p, y[idx:idx+bs])
        loss.backward()
        opt.step()
        # keep track of some summary statistics
        eloss['loss'].append(loss.item())
        eloss['n'].append(x[idx:idx+bs].size(0))
        eloss['corr'].append(sum(p.max(axis=1)[1] == y[idx:idx+bs]))
    print(f'epoch {e:3d} avg loss; {sum(eloss["loss"])/sum(eloss["n"]):.4f}')
    print(f'        acc.;    {sum(eloss["corr"])/sum(eloss["n"]):.4f}')
print("done")

net.eval()
with torch.no_grad():
    final_accuracy = (net(x).argmax(dim=1) == y).sum().item() / len(y)
print(f"Final accuracy: {final_accuracy:.4f}")

epoch   1 avg loss; 174.5849
        acc.;    0.2624
epoch   2 avg loss; 38.5660
        acc.;    0.4255
epoch   3 avg loss; 45.6517
        acc.;    0.0567
epoch   4 avg loss; 11.3229
        acc.;    0.4752
epoch   5 avg loss; 34.0411
        acc.;    0.2979
epoch   6 avg loss; 18.9215
        acc.;    0.3333
epoch   7 avg loss; 33.1146
        acc.;    0.2837
epoch   8 avg loss; 23.2495
        acc.;    0.1844
epoch   9 avg loss; 19.5408
        acc.;    0.3617
epoch  10 avg loss; 31.0224
        acc.;    0.1348
epoch  11 avg loss; 26.9013
        acc.;    0.3050
epoch  12 avg loss; 26.8348
        acc.;    0.2270
epoch  13 avg loss; 14.8363
        acc.;    0.3617
epoch  14 avg loss; 29.2539
        acc.;    0.2908
epoch  15 avg loss; 19.1892
        acc.;    0.3050
epoch  16 avg loss; 36.3302
        acc.;    0.2482
epoch  17 avg loss; 18.0741
        acc.;    0.2199
epoch  18 avg loss; 25.2936
        acc.;    0.2979
epoch  19 avg loss; 25.5303
        acc.;    0.1135
epoch  20 a

In [21]:
_, test_mean = preprocessing(test, test_label)

tmp_mean_target = test_mean['target']

tmp_test = test_mean[feature].copy()
test_x = torch.tensor(tmp_test.values).float()
test_y = torch.tensor(tmp_mean_target.values - 1).long()

net.eval()

with torch.no_grad():
    predictions = net(test_x)
    predicted_labels = predictions.max(dim=1)[1]
    
    accuracy = (predicted_labels == test_y).sum().item() / len(test_y)
    print(f'Test Accuracy: {accuracy:.4f}')

    test_loss = los(predictions, test_y)
    print(f'Test Loss: {test_loss.item():.4f}')

Test Accuracy: 0.2490
Test Loss: 262.0643


In [22]:
X_all = train_all
X_all.drop(columns=['number'], inplace=True)
X_all_target = X_all['target']
X_all.drop(columns=['target'], inplace=True)

print(X_all.values.shape)
print(X_all_target.shape)

data_for_logit_all = X_all[feature].copy()
data_for_logit_all


(9705, 27)
(9705,)


,sleep_light,sleep_score_latency,sleep_midpoint_time,sleep_duration,sleep_onset_latency,sleep_restless,sleep_score_alignment,sleep_total,sleep_score_deep,sleep_awake
0,11670,94,12390,24480,990,32,99,22620,92,1860
1,11250,99,15750,31200,900,22,100,29340,100,1860
2,14100,97,15270,30420,840,31,100,27210,97,3210
3,15750,91,14520,28980,690,32,100,27090,96,1890
4,18600,72,14940,30180,300,33,100,28980,81,1200
...,...,...,...,...,...,...,...,...,...,...
9700,20580,94,23640,46200,780,42,100,36720,75,9480
9701,18030,99,20340,38280,900,35,100,30000,70,8280
9702,13470,72,13350,26940,300,42,100,24360,95,2580
9703,18420,97,19620,40860,930,39,100,31920,91,8940


In [23]:
x = torch.tensor(data_for_logit_all.values).float()
y = torch.tensor(X_all_target.values-1).long()

In [24]:
from collections import defaultdict
 
bs = 32
for e in range(1,21):
    eloss = defaultdict(lambda: [])
    for idx in range(0,len(x),bs):
        opt.zero_grad()
        p = net(x[idx:idx+bs])
        loss = los(p, y[idx:idx+bs])
        loss.backward()
        opt.step()
        # keep track of some summary statistics
        eloss['loss'].append(loss.item())
        eloss['n'].append(x[idx:idx+bs].size(0))
        eloss['corr'].append(sum(p.max(axis=1)[1] == y[idx:idx+bs]))
    print(f'epoch {e:3d} avg loss; {sum(eloss["loss"])/sum(eloss["n"]):.4f}')
    print(f'        acc.;    {sum(eloss["corr"])/sum(eloss["n"]):.4f}')
print("done")

net.eval()
with torch.no_grad():
    final_accuracy = (net(x).argmax(dim=1) == y).sum().item() / len(y)
print(f"Final accuracy: {final_accuracy:.4f}")

epoch   1 avg loss; 5.8574
        acc.;    0.9520
epoch   2 avg loss; 60.7456
        acc.;    0.8752
epoch   3 avg loss; 32.5762
        acc.;    0.8485
epoch   4 avg loss; 14.9873
        acc.;    0.8460
epoch   5 avg loss; 4.3712
        acc.;    0.8486
epoch   6 avg loss; 1.8047
        acc.;    0.8453
epoch   7 avg loss; 0.7068
        acc.;    0.8137
epoch   8 avg loss; 0.1362
        acc.;    0.6295
epoch   9 avg loss; 0.0505
        acc.;    0.6851
epoch  10 avg loss; 0.0415
        acc.;    0.7723
epoch  11 avg loss; 0.0375
        acc.;    0.6102
epoch  12 avg loss; 0.0703
        acc.;    0.6629
epoch  13 avg loss; 0.0319
        acc.;    0.6114
epoch  14 avg loss; 0.0303
        acc.;    0.6121
epoch  15 avg loss; 0.0291
        acc.;    0.6182
epoch  16 avg loss; 0.0283
        acc.;    0.6188
epoch  17 avg loss; 0.0281
        acc.;    0.6120
epoch  18 avg loss; 0.0281
        acc.;    0.6080
epoch  19 avg loss; 0.0276
        acc.;    0.6094
epoch  20 avg loss; 0.0278
 

In [27]:
test_all, test_mean = preprocessing(test, test_label)

test_all_target = test_all['target']

X_test_all = test_all[feature].copy()
test_x_all = torch.tensor(X_test_all.values).float()
test_y_all = torch.tensor(test_all_target.values - 1).long()

net.eval()

with torch.no_grad():
    predictions = net(test_x_all)
    predicted_labels = predictions.max(dim=1)[1]
    
    accuracy = (predicted_labels == test_y_all).sum().item() / len(test_y_all)
    print(f'Test Accuracy: {accuracy:.4f}')

    test_loss = los(predictions, test_y_all)
    print(f'Test Loss: {test_loss.item():.4f}')

Test Accuracy: 0.7879
Test Loss: 0.8136
